In [1]:
import pandas as pd
import altair as alt
url = 'https://raw.githubusercontent.com/xh0417/IV_group63/main/europe_covid.csv'
data = pd.read_csv(url)

In [2]:
# Select the top 10 countries with the highest number of active cases
attribute = ['Top-10 Active Cases', 'Top-10 Total Cases', 'Top-10 Total Deaths']

top10_active = pd.DataFrame({
    'Country': data.nlargest(10, 'Active Cases')['Country'],
    'Value': data.nlargest(10, 'Active Cases')['Active Cases'],
    'Attribute': 'Top-10 Active Cases'
})

top10_TotalCases = pd.DataFrame({
    'Country': data.nlargest(10, 'Total Cases')['Country'],
    'Value': data.nlargest(10, 'Total Cases')['Total Cases'],
    'Attribute': 'Top-10 Total Cases'
})

top10_TotalDeaths = pd.DataFrame({
    'Country': data.nlargest(10, 'Total Deaths')['Country'],
    'Value': data.nlargest(10, 'Total Deaths')['Total Deaths'],
    'Attribute': 'Top-10 Total Deaths'
})
all_stats = pd.concat([top10_active, top10_TotalCases, top10_TotalDeaths], axis=0)

In [3]:
def calPercent(data,names):
    for i in names:
        col_sum = data[i].sum()
        result_data = data[i].apply(lambda x: x / col_sum)
        data[i] = result_data
    return data
data = calPercent(data,["Total Cases","Total Deaths", "Total Recovered", "Active Cases", "Tot Cases/ 1M pop","Deaths/ 1M pop","Total Tests","Tests/ 1M pop","Population"])
df = pd.melt(data, id_vars=["Country"], value_vars=["Total Cases","Total Deaths","Total Recovered","Active Cases","Tot Cases/ 1M pop","Deaths/ 1M pop","Total Tests","Tests/ 1M pop","Population"], var_name="attribute", value_name="value")

In [4]:
brush = alt.selection_single(
    encodings=['y'],
    on='mouseover'
)
selection = alt.selection_single(
    name='Select',
    fields=['Attribute'],
    init={'Attribute': 'Top-10 Active Cases'},
    bind={'Attribute': alt.binding_radio(options=attribute)}
)

In [5]:
bars = alt.Chart().mark_bar().encode(
    x='Value:Q',
    y=alt.Y('Country:N', sort='-x'),
    color=alt.Color("Country:N")
).properties(
    width=400, height=300,
    title='COVID-19 Statistics by Country'
)
# Filter the data based on the select box value
bars_filtered = bars.transform_filter(
    selection
).properties(
    title='COVID-19 Statistics by Country'
)

In [6]:
# Combine the chart and data
chart = alt.layer(
    bars_filtered,
    data=all_stats
).add_selection(
    selection,brush
)

chart2 = alt.Chart(df).mark_bar().encode(
    x=alt.X("attribute:N"),
    y=alt.Y("value:Q"),
    color=alt.Color("Country:N"),
    opacity=alt.condition(brush, alt.value(0.75), alt.value(0.05))
).properties(width=400, height=300)
br1 = alt.hconcat(chart, chart2).properties(spacing=5)
br1.save('brush_linking1.html')